$$Exercise: 3$$

#Q-1

$The\; company \;feels \;that\; a\;
linear\; model \;of \;the \;form\; P = b_0 +b_1L+b_2E \;would\; be\; reasonably\; accurate \;and \;easy\; to \;use.\; Here,\; b_1\;and \;b_2\; indicate\; how \;the\; price \;varies \;with \;lot\; size\; and \;elevation, \;respectively, \;while \;b_0 \;would\;
reflect \;a\; base\; price\; for\; this \;section \;of\; the\; city. \;Therefore, the\; optimization\; model\; will\; be \;as \;folllows:$ \\
$$\text{min} \ \  \sum_{i=1}^{6} |P_i-\tilde{P_i}| \\
\text{Subject to:  } \\
\tilde{P_i}=b_0+b_1L_i+b_2E_i \ \ \forall i  \\
b_0 \geq 0$$ \\

\\
$No,\; the\; above \;optimization \;model\; is\; not \;linear\; since,\; it\; contains\; the\; modulus \;function,\; which \;is\; piecewise\; but \;not\; linear.\; But,\;  Instead\; of \;minimizing \;the\; absolute\; value\; of\; the \;residual,\; |P_i-\tilde{P_i}|, \;we\; can\; also\; instead\; minimize\;the\; square,\; i.e. \;(P_i-\tilde{P_i})^2$ \\
$Therefore,\; the\; optimization\; problem \;will\; be\; as\; follows:$

$$\text{min} \ \  \sum_{i=1}^{6} (P_i-\tilde{P_i})^2 \\
\text{Subject to:  } \\
\tilde{P_i}=b_0+b_1L_i+b_2E_i \\
b_0 \geq 0$$

In [19]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

removed 'ipopt'
Reading package lists...
Building dependency tree...
Reading state information...
libgfortran5 is already the newest version (8.4.0-1ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [20]:
!./ipopt test.nl



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      230

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

In [21]:
!pip install -q pyomo 

In [22]:
from pyomo.environ import *

In [23]:
import numpy as np

In [24]:
import pandas as pd

In [25]:
data_csvfile = pd.read_csv('file41.csv')

In [26]:
model = ConcreteModel()

In [27]:
N = len(data_csvfile.index)
n = 3 
M = 2 
col_indices = np.arange(N)
b_indices = np.arange(n)
row_indices = np.arange(M)

In [28]:
model.b = Var(b_indices)
expres=0
for i in col_indices: 
  expres=expres+(model.b[0]+model.b[1]*data_csvfile['lot size'][i]+model.b[2]*data_csvfile['elevation'][i] - data_csvfile['seling price'][i])**2

In [29]:
model.objective = Objective(expr=expres,sense=minimize)

In [30]:
model.constraints = ConstraintList()

In [31]:
model.b[0].setlb(0)

In [32]:
model.pprint()

2 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}

1 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (b[0] + 1800*b[1] + 250*b[2] - 175000)**2 + (b[0] + 2800*b[1] + 400*b[2] - 150000)**2 + (b[0] + 3500*b[1] + 450*b[2] - 125000)**2 + (b[0] + 800*b[1] + 100*b[2] - 90000)**2 + (b[0] + 700*b[1] + 200*b[2] - 80000)**2 + (b[0] + 500*b[1]

In [33]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model)
print(result)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 3
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.022820711135864258
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



#Q-2

$Reporting \;optimal\; solution\; values \;for\;the\; original\; and \;new\; decision \;variables,\;and \;the\; optimal\; sum \;of\; residuals$

In [34]:
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.objective())

Solver status: ok
Solver termination condition: optimal

Objective =  4291134028.8074555


In [35]:
print('\nOptimal sum of residuals = ', model.objective())

print('\nOriginal Decision Variables:')
for i in b_indices:
  print('b[',i,']=', model.b[i].value)


Optimal sum of residuals =  4291134028.8074555

Original Decision Variables:
b[ 0 ]= 104904.4164371955
b[ 1 ]= 38.702075831391646
b[ 2 ]= -193.3223893242956


In [36]:
for i in col_indices:
  print('Deviation[',i,']=',abs(data_csvfile['seling price'][i]- model.b[0].value-model.b[1].value*data_csvfile['lot size'][i] -model.b[2].value*data_csvfile['elevation'][i] ))

Deviation[ 0 ]= 48762.444397373445
Deviation[ 1 ]= 14058.726964626141
Deviation[ 2 ]= 28366.606651133246
Deviation[ 3 ]= 26533.838169879255
Deviation[ 4 ]= 13331.391654310522
Deviation[ 5 ]= 5410.665113323461


Here, the maximum deviation is: 48762.444397373445
And in the previous model(Lab06), the maximum deviation is: 54272.727

Standard Deviation gives more weightage to the extreme value and hence is considered a better estimation than Mean Absolute Deviation. This is the reason why this model have a lower maximum value as compared to the other(Lab06).